# Pandas for beginners

![pandas](img/pandas.png)

Useful links
<br>➡ Documentation : [pandas.pydata.org](https://pandas.pydata.org/docs/index.html)

Pandas is a sort of plugin to Python. We will still use the basic Python logic in many cases, but in many ways Pandas also works completely different.

# 1. Intro into data types

In the previous lesson, you saw strings, numbers, `lists` and `dictionaries`. These are data types. Pandas works with 2 additional data types: `Series` and `DataFrames`

In [15]:
import pandas as pd

## Series

`Series` are akin to columns of a table. You can create a `Series` from a `list`

In [7]:
s = pd.Series([1, 3, 5, 7, 6, 8])
s

0    1
1    3
2    5
3    7
4    6
5    8
dtype: int64

## DataFrames

`DataFrame` is an actual table with rows (index) and columns. There are many ways to create a DataFrame. We will be creating them exclusively from excel sheets and CSV tables, so this is just to understand how a dataframe is structured

In [31]:
from random import randint

df = pd.DataFrame({
        "A": 100.0,
        "B": pd.Timestamp("20130102"),
        "C": [randint(30,60), randint(30,60), randint(30,60), randint(30,60)],
        "D": [randint(30,60), randint(30,60), randint(30,60), randint(30,60)],
        "E": ["test", "train", "test", "train"],
        "F": "whatever"}
)
df

,A,B,C,D,E,F
0,100.0,2013-01-02,36,30,test,whatever
1,100.0,2013-01-02,60,56,train,whatever
2,100.0,2013-01-02,57,59,test,whatever
3,100.0,2013-01-02,58,34,train,whatever


In [37]:
df = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]], index=[0,1,2], columns=['A','B','C'])
df

,A,B,C
0,1,2,3
1,4,5,6
2,7,8,9


### Questions:
* From what other data types is the DataFrame structured in these two cases?

# Opening and exploring data

In [1]:
# We need to import the library first.
import pandas as pd

# and set some options for the max columns and number format
pd.options.display.max_columns = 100
pd.options.display.float_format = '{:,.2f}'.format

We will be working with the most recent published year of EU spending (2021), using the data from [EU Financial Transparency System (FTS)](https://ec.europa.eu/budget/financial-transparency-system/index.html). You will find it in the `data` folder

Depending on your data source, you can use either of these functions

```python
df = pd.read_csv()
df = pd.read_excel()
```

## 1. Open

In [2]:
df = pd.read_csv("data/FTS_2021.csv", na_values="-")

## 2. First exploration 

The first step for a data analysis is to look at the data first. What columns are there? How many rows? What is in the columns? Pandas has very helpful methods to do this.

```python
df.info() # information on the column names and non-zero values
df.sample(3) # returns 3 sample rows from the data
df.head(3) # returns the first 3 rows of the data
len(df) # returns the number of rows
df.columns # all the column names as a list
```
Try them all!

After a first look, we want to see unique values for some of the columns, and visualize the numeric columns. We isolate columns using this syntax:

```python
df["column"]
```

This will return a `Series` from a `DataFrame`. It's important to know what data type you are working with, as all of them have their own methods.


```python
# for categorical variables
df["column"].unique() # all the unique values of the column
df["column"].value_counts() # how often does a value occur

# for numeric variables
df.hist(column='column') # makes a histogram
df.describe() # descriptive statistics for all numeric variables
df["column"].describe() # descriptive statistics for a single column
```

Explore the dataset

We could also use what we learned about loops and let python analyze some of the data for us.

```python
for col in df.columns:
    print(f"{col} : {len(df[col].unique())}")
```

Let's paste this below and dissect it

### 2.1 Beware of the duplicates

## 3. Clean

Data cleaning is a very disliked but important step. In our dataset, the numeric columns contain a comma as thousands separators.<br>Therefore, Pandas does not recognize the number as a number, but as a string. See also the data type next to the columns in `df.info()`.

First, we will use `.str.replace()` to replace the comma with nothing. Then we will use `pd.to_numeric()` to make numeric column from a string.<br>
We will save the output in a new column. To create a new column, we will use the same syntax as for isolating a column, but assigning a value to it first.
```python
df["beneficiary_contracted_amount"] = pd.to_numeric(df["Beneficiary’s contracted amount (EUR)"].str.replace(",",""))
```

Another step of cleaning we might want to do is filtering the data to only contain columns we want.

```python
# make a list of column names we would like to keep
keep = ["Name of beneficiary", "beneficiary_contracted_amount", "Beneficiary country", "Beneficiary type" ,"Subject of grant or contract", "Programme name"]

# make a copy of a the DataFrame and use the same brackets as with selecting one column []
filtered = df.copy()[keep]
```

## 4. More exploration 

Let's first find out what is the largest award in our dataset. We will sort the table based on the newly created `beneficiary_contracted_amount` variable.

```python
filtered.nlargest(30, "beneficiary_contracted_amount")
```

The largest values are Recovery and Resilience disimbursement to member states. We are mostly interested in private companies though. We can filter them out like this:

```python
filtered[filtered["Beneficiary type"] == "Private Companies"]
```

We can even save them as another separate DataFrame

```python
private_companies = filtered[filtered["Beneficiary type"] == "Private Companies"]
```

In [6]:
private_companies = filtered[filtered["Beneficiary type"] == "Private Companies"]
private_companies.head()

,Name of beneficiary,beneficiary_contracted_amount,Beneficiary country,Beneficiary type,Subject of grant or contract,Programme name
0,*****,0.00,Afghanistan,Private Companies,*****,"6.0.111 - Neighbourhood, Development and Inter..."
1,*****,"600,000.00",Russia,Private Companies,*****,"6.0.111 - Neighbourhood, Development and Inter..."
2,*****,"600,000.00",Russia,Private Companies,*****,"6.0.111 - Neighbourhood, Development and Inter..."
3,*****,"600,000.00",Russia,Private Companies,*****,"6.0.111 - Neighbourhood, Development and Inter..."
4,*****,"580,000.00",Russia,Private Companies,*****,"6.0.111 - Neighbourhood, Development and Inter..."


Now that we have a numeric column to explore, let's use two ways to group our data:

```python
df.groupby("column").agg() # aggregate based on 1 or more columns (more columns reqire making a list)
pd.pivot_table(df, index="column1", columns="column2", values="column3", aggfunc="sum") # will create a pivot table with a column in the rows and in the columns
```
So let's try:

```python
private_companies.groupby("Name of beneficiary").sum()
```

This will sum up the money acquired by a particular beneficiary. Instead of `.sum()` we can use for example `.count()`, `.mean()`, `.median()`, `.min()`, `.max()`. You can also use multiple of these at the same time, using the `.agg()` function. Google how that works.


Pivot tables are similar to groupby, but they have rows and columns.

```python
pd.pivot_table(filtered, index="Beneficiary country", columns="Beneficiary type", values="beneficiary_contracted_amount", aggfunc="sum")
```

## 5.Data visualization

In [7]:
import altair as alt

You can find lots of examples on how to make data visualizations with altair on [the documentation example gallery](https://altair-viz.github.io/gallery/index.html)

Making a simple bar chart of the top 5 largest recipients:

```python
chart_data = filtered.nlargest(5, "beneficiary_contracted_amount")

alt.Chart(chart_data).mark_bar().encode(
       y = alt.Y("Name of beneficiary", sort="-x"),
       x = "beneficiary_contracted_amount"
)
```

## 6. Conditional formatting

To highlight the largest values of a table, pandas offers us the possibility to use styling on the dataset.

```python
pivot = pd.pivot_table(filtered, index="Beneficiary country", columns="Beneficiary type", values="beneficiary_contracted_amount", aggfunc="sum")
pivot[["Private Companies", "Private persons" ,"Public Bodies"]].nlargest(30, 'Private Companies').style.background_gradient()
```

# Merging data

Let's combine the data with another dataset. This is the where we can get unique insights. We will combine the Financial Transparency System data with the Transparency System data (lobby register).

```python
df2 = pd.read_excel("data/TransparencyRegister.xls")
df2.sample(3)
```

We need to find a column in both datasets that is the same. Which one would that be?

There are disparities in the case of the values of this column. Let's fix that first.

Now let's `merge` and have a look at the data

```python
pd.merge(df1, df2, left_on="column", right_on="column", how="inner")
```
Merge possibilities: 
* `inner` - new data has only rows that contain matches
* `outer` - new data contains all rows from both data sets
* `left` - new data contains all rows from the 'left' data set
* `right` - guess

Let's use `groupby` to get some more information

## Beware of duplicates especially while merging

In the FTS, a company can get multiple contracts in a year. That means that it can appear in multiple rows in the data. In the Transparency Register, there is only one row per institution.